In [ ]:
pip install ragas


INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.5 MB/

In [ ]:
import os
import openai
from langchain.chat_models import ChatOpenAI

# 환경 변수 설정
os.environ["OPENAI_API_KEY"] = ""

# OpenAI 모델 생성
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


<ipython-input-2-b9280030d095>:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [ ]:
import os
import json

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)
from datasets import Dataset

# RAGAS 평가를 위한 평가 지표 설정
metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

# JSON 파일 경로 설정
file_path = '/content/gemini_fewshot.json'

# JSON 파일 읽어오기
with open(file_path, 'r', encoding='utf-8') as file:
    qa_dataset = json.load(file)

# 데이터 확인
for entry in qa_dataset:
    print(f"CONTEXT: {entry['CONTEXT']}")
    print(f"QUESTION: {entry['QUESTION']}")
    print(f"ANSWER: {entry['ANSWER']}\n")

# 평가를 위해 필요한 필드 추출
test_questions = [entry["QUESTION"] for entry in qa_dataset]
test_groundtruths = [entry["ANSWER"] for entry in qa_dataset]
contexts = [[entry["CONTEXT"]] for entry in qa_dataset]

# 평가 데이터셋 생성
response_dataset = Dataset.from_dict({
    "question": test_questions,
    "answer": test_groundtruths,  # 이 예제에서는 정답을 answer로 대체
    "contexts": contexts,
    "ground_truth": test_groundtruths
})

# dataset 첫 번째 데이터 확인
print(response_dataset[0])

# RAGAS 평가 수행
evaluation_results = evaluate(response_dataset, metrics, raise_exceptions=False)

# 평가 결과 출력
print("RAGAS 평가 결과:")
for metric_name, score in evaluation_results.items():
    print(f"{metric_name}: {score:.4f}")

CONTEXT: ▶ 선박평형수 신고 
- 관할수역외의 수역에서 선박평형수를 주입한 후 관할수역에 들어오는 선박 대상
- 선박평형수 및 그 침전물을 효과적으로 처리·교환·주입·배출에 관한 사항을 신고
- 일본 동부 6개현(아오모리현, 이와테현, 후쿠시마현, 미야기현, 이바라기현, 치바현)에서 평형수를 주입 후 
   ≫ 국내 입항 예정인 선박 : 국내 입항 24시간 전까지 평형수 입항보고서 제출
   ≫ 국내 배출 예정인 선박 : 우리나라 관할수역 밖에서 평형수 교환 후 입항
▶ 선박보안정보통보 신고
-  국제항해에 이용되는 선박과 그 선박이 이용하는 항만시설의 보안에 관한 사항을 신고
-  신고대상 선박(외국적의 모든 여객선 및 총톤수 500톤 이상 화물선, 이동식 해상구조물)
선박평형수 : 선박평형수 관리법 제5조, 시행규칙 제10조
선박보안정보통보 : 국제선박항만보안법 제19조, 시행규칙 제20조 
근거
법령
민원 신고 
완료
• 선박평형수 및 선박보안정보통보 신고 완료8
외항선 
입항 신고
• 선박 입항 전까지 입항 예정정보를 신고(최초)
• 실제 선박 입,출항 후 (최종)신고
2
(담당자)수리
 •  업무담당자: 민원 신고문서 심사 후 이상이 없으면 
수리처리
7
오류통보
(반려)
6
•  업무담당자 : 민원 신청
내용에 오류가 있는  
경우 오류내용 입력 후 
(오류)처리
•  민원인 : 오류 내용  
확인 후 신청정보를  
수정하여 재신고
신고 문서검토 
및 저장
• 신고내역 확인 및 저장
• 담당자 반려시 반려에 대한 내용 확인 후 재신고
4
�
QUESTION: 선박평형수 신고는 어떤 선박을 대상으로 이루어지나요?
ANSWER: 선박평형수 신고는 관할수역외의 수역에서 선박평형수를 주입한 후 관할수역에 들어오는 선박을 대상으로 합니다.

CONTEXT: ▶ 선박평형수 신고 
- 관할수역외의 수역에서 선박평형수를 주입한 후 관할수역에 들어오는 선박 대상
- 선박평형수 및 그 침전물을 효과적으로 처리·교환·주입·배출에 관한 사항을 신고
- 일본 동부 

Evaluating:   0%|          | 0/325 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[37]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-HSIwTeKyjsTVKUBmhyLkjKsk on tokens per min (TPM): Limit 200000, Used 198270, Requested 2050. Please try again in 96ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[112]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-HSIwTeKyjsTVKUBmhyLkjKsk on tokens per min (TPM): Limit 200000, Used 199708, Requested 1973. Please try again in 504ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[88]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-HSI

RAGAS 평가 결과:
faithfulness: 0.3705
answer_relevancy: 0.0856
context_recall: 0.8579
context_precision: 0.9219
answer_correctness: 0.7695
